# VI. Types, type hierarchy, and multiple dispatch

In Julia types play a central role. We've covered enough so far that you can write Julia code as you would in other languages, but having an understanding of types will give you a better understanding of how Julia works and hopefully help you write better Julia code. 

In Julia, everything has a type. As we've already seen, you can figure out an object's type using the __typeof__ funciton.

In [1]:
typeof(1.3)

Float64

In [2]:
typeof(22)

Int64

In [3]:
typeof("julia")

String

In [4]:
typeof(3//2)

Rational{Int64}

In [5]:
typeof(3 + 2im)

Complex{Int64}

In [6]:
typeof(randn(5, 5))

Array{Float64,2}

The curly braces indicate the type is a **parametric** type. So *Array* is a parametric type since it's a type with type parameters (in this case *Float64* and *2*).

You can use the __eltype__ function to determine the element type.

In [7]:
A = rand(5, 5)

5×5 Array{Float64,2}:
 0.0297536  0.808987  0.898289   0.0597706  0.91027 
 0.501498   0.836653  0.624897   0.780903   0.186859
 0.357212   0.254329  0.300162   0.192892   0.835316
 0.575208   0.380088  0.0502164  0.187423   0.664723
 0.448893   0.251424  0.945135   0.806016   0.30771 

In [8]:
eltype(A)

Float64

In [9]:
t = ("summer", 32, 4.3)

("summer", 32, 4.3)

In [10]:
typeof( t )

Tuple{String,Int64,Float64}

In [11]:
eltype(t)

Any

Because of Julia's emphasis on types you can sometimes run into unexpected problems. Let's generate a 5x5 random array of integers between 1 and 20:

In [12]:
A = rand(1:20, 5, 5)

5×5 Array{Int64,2}:
 20   2   9   8  10
  1  13   7  14  11
  9   8  12   5  13
  3  15  18  13  13
 19  18   6  14   1

As stated before, arrays are mutable:

In [13]:
A[1, 1] = 19

19

In [14]:
A

5×5 Array{Int64,2}:
 19   2   9   8  10
  1  13   7  14  11
  9   8  12   5  13
  3  15  18  13  13
 19  18   6  14   1

Let's now set element A[1,2] equal to 3.2:

In [15]:
A[1, 2] = 3.2

InexactError: InexactError: Int64(3.2)

What happened here?

### How types are organized in Julia:

In Julia, types are organized according according to a tree like  structure. At the top of the tree is the *Any* type. So all types are subtypes of *Any*. At the bottom of the tree (the leaves of the tree) are __concrete types__, i.e. *Float64*, *Int64*, *UInt32*, etc. A type can have at most one parent, but possibly more than one child.

To get a sense of how the type hierarchy is organized you can use the functions __subtypes__ and __supertype__. Let's start with a simple concrete type *Float64* and traverse the tree upwards.

In [16]:
supertype(Float64)

AbstractFloat

So the parent of *Float64* is a type called *AbstractFloat* which in Julia is called an **abstract type**.

In [17]:
supertype(AbstractFloat)

Real

In [18]:
supertype(Real)

Number

In [19]:
supertype(Number)

Any

In [20]:
supertype(Any)

Any

From the above ouptut, we see that *Real*, *Number*, etc. are also **abstract types** (i.e. they can have children). When you create variables in Julia they can only be __concrete types__, i.e. you can not instantiate a variable of type *Real*.

Let's now start with the *Real* type and use the **subtypes** function to go down the tree.

In [21]:
subtypes(Real)

4-element Array{Any,1}:
 AbstractFloat     
 AbstractIrrational
 Integer           
 Rational          

So *Real* has four subtypes: *AbstractFloat*, *Integer*, *Irrational*, *Rational*.

In [22]:
subtypes(Integer)

3-element Array{Any,1}:
 Bool    
 Signed  
 Unsigned

In [23]:
subtypes(Signed)

6-element Array{Any,1}:
 BigInt
 Int128
 Int16 
 Int32 
 Int64 
 Int8  

In [24]:
subtypes(Int64)

0-element Array{Type,1}

We can see that *Int64* is a **concrete type**. There are also suptype and supertype operators you can use to ask questions if a type is a supertype or subtype of some other type. Here we test if *Real* is a subtype of *Number*:

In [25]:
Real <: Number

true

Is *Int64* a subtype of *Float64*? Remember these are both concrete types.

In [26]:
Int64 <: Float64

false

Let's test if *Real* is a supertype of *Rational*:

In [27]:
Real >: Rational

true

Is *Int64* a supertype of *Complex*?

In [28]:
Int64 >: Complex

false

Types are important in Julia for many reasons, but perhaps most obviously due to Julia's approach to calling functions known as **multiple dispatch**.

In Julia many versions of the same named function can exist and these different versions are called **methods**. The methods differ in that they have different function signatures. In general, these different methods correspond to a specific type of function behavior determined by the type of inputs.

You can see this with the built-in multiplication function __*__:

In [29]:
*(3, 2.2)

6.6000000000000005

In [30]:
*("string ","input")

"string input"

As we can see the behavior of __*__ is different depending on the input types: for numbers it does multiplication and for strings it does concatenation. You can see all the methods for a function using **methods**:

In [31]:
methods( + )

# 166 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:282
[2] +(x::Bool, y::Bool) in Base at bool.jl:96
[3] +(x::Bool) in Base at bool.jl:93
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:104
[5] +(x::Bool, z::Complex) in Base at complex.jl:289
[6] +(a::Float16, b::Float16) in Base at float.jl:398
[7] +(x::Float32, y::Float32) in Base at float.jl:400
[8] +(x::Float64, y::Float64) in Base at float.jl:401
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:283
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:297
[11] +(::Missing, ::Missing) in Base at missing.jl:114
[12] +(::Missing) in Base at missing.jl:100
[13] +(::Missing, ::Number) in Base at missing.jl:115
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:413
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:503
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:502
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:501
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:472
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:509
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:515
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:563
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:556
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:550
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:409
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:378
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:385
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:393
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:401
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:76
[43] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling)

What Julia does at runtime is call the specialized version of the function (i.e. method) that corresponds to the number of input arguments and types of all input arguments being passed. This process is what is referred to as multiple dispatch.

For your own functions you can incorporate type annotations, e.g.

In [32]:
function PlusTwoSpecific(x::Float64)
    return x + 2
end

PlusTwoSpecific (generic function with 1 method)

In [33]:
function PlusTwoSpecific(x::Int64)
    return x + 2
end

PlusTwoSpecific (generic function with 2 methods)

In [34]:
methods(PlusTwoSpecific)

# 2 methods for generic function "PlusTwoSpecific":
[1] PlusTwoSpecific(x::Int64) in Main at In[33]:2
[2] PlusTwoSpecific(x::Float64) in Main at In[32]:2

However it is worth noting that Julia will do type inference on the argument being passed in. The first time it encounters this argument type it will infer the argument type, then compile and cache this compiled version of the function for the inferred argument type. The next time the function is called with the same argument type Julia will call the specialized compiled version that it has stored in memory.

Therefore type annotations are not always needed and in many cases you can use a generic function signature:

In [35]:
function PlusTwo( x )
    return x + 2
end

PlusTwo (generic function with 1 method)

Type annotations in your functions may or may not improve the performance of your code and you want to avoid overspecialization of your functions. Type annotations are often used to guarantee functions behave a specific way for certain input types.

For example, adding two generically works for *Int64* and *Float64* numeric types, so there is no need to create two separate methods of __PlusTwo__ to deal with each of these types. However, what if I wanted my __PlusTwo__ function to work on say a __Date__ type? 

In [36]:
using Dates

myDate = Date(2021,5,21)

2021-05-21

In [37]:
PlusTwo(myDate)

MethodError: MethodError: no method matching +(::Date, ::Int64)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  +(!Matched::Complex{Bool}, ::Real) at complex.jl:297
  +(!Matched::Missing, ::Number) at missing.jl:115
  ...

In this case, adding two to a __Date__ type doesn't generically work and the behavior of __PlusTwo__ for this input type needs to defined.

In [38]:
function PlusTwo(x::Date)
    return x + Day(2)
end

PlusTwo (generic function with 2 methods)

In [39]:
PlusTwo(myDate)

2021-05-23

# Exercise 6
* Find the supertype of the Bool type.
* Is Int64 subtype of Bool?
* Display all the methods for the "isless" function.

In this lesson we covered:
* Julia types
* Type operations
* Mulitple dispatch